<a href="https://colab.research.google.com/github/malaknabil712002/linear_regression/blob/main/house_prices_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [218]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

In [219]:
#import data and remove useless columns
train=pd.read_csv('train.csv')
train.drop(columns=["Id"],inplace=True)


In [236]:
#making dummy variable to replace strings with numerical values
train_new=pd.get_dummies(train,columns=['SaleCondition','SaleType','MiscFeature','Fence','PoolQC','PavedDrive','GarageCond','GarageQual','GarageFinish','FireplaceQu','GarageType','Functional','KitchenQual','HeatingQC','CentralAir','Electrical','Heating','BsmtFinType2','Exterior1st','Exterior2nd','MasVnrType','MasVnrArea','ExterQual','ExterCond','Foundation','BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1','Condition1','Condition2','BldgType','HouseStyle','OverallQual','OverallCond','YearBuilt','YearRemodAdd','RoofStyle','RoofMatl','Street','Alley','LotShape','LandContour','Utilities','LotConfig','LandSlope','Neighborhood','MSZoning'],drop_first=True)
train_new.info()
train_new.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Columns: 755 entries, MSSubClass to MSZoning_RM
dtypes: float64(2), int64(30), uint8(723)
memory usage: 1.4 MB


,MSSubClass,LotFrontage,LotArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,...,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker,MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM
0,60,65.0,8450,706,0,150,856,856,854,0,...,0,0,0,0,0,0,0,0,1,0
1,20,80.0,9600,978,0,284,1262,1262,0,0,...,0,0,0,0,0,1,0,0,1,0
2,60,68.0,11250,486,0,434,920,920,866,0,...,0,0,0,0,0,0,0,0,1,0
3,70,60.0,9550,216,0,540,756,961,756,0,...,0,0,0,0,0,0,0,0,1,0
4,60,84.0,14260,655,0,490,1145,1145,1053,0,...,0,0,0,0,0,0,0,0,1,0


In [221]:
#missing data
missing_values=train_new.isnull().sum()
missing_values.sort_values(ascending=False)

LotFrontage         259
GarageYrBlt          81
Condition2_PosA       0
Condition2_RRAe       0
Condition2_RRAn       0
                   ... 
MasVnrArea_183.0      0
MasVnrArea_184.0      0
MasVnrArea_186.0      0
MasVnrArea_188.0      0
MSZoning_RM           0
Length: 755, dtype: int64

In [237]:
#handling missing data
train_new.drop(columns=train_new.columns[train_new.isnull().sum().values>200],inplace=True)
train_new['GarageYrBlt']=train_new['GarageYrBlt'].fillna(train_new['GarageYrBlt'].mean())
train_new.isnull().sum()

MSSubClass              0
LotArea                 0
BsmtFinSF1              0
BsmtFinSF2              0
BsmtUnfSF               0
                       ..
Neighborhood_Veenker    0
MSZoning_FV             0
MSZoning_RH             0
MSZoning_RL             0
MSZoning_RM             0
Length: 754, dtype: int64

In [223]:
#Adding bias column
train_new["bias"] = np.ones(train_new.shape[0])
train_new.head()

,MSSubClass,LotArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,...,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker,MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,bias
0,60,8450,706,0,150,856,856,854,0,1710,...,0,0,0,0,0,0,0,1,0,1.0
1,20,9600,978,0,284,1262,1262,0,0,1262,...,0,0,0,0,1,0,0,1,0,1.0
2,60,11250,486,0,434,920,920,866,0,1786,...,0,0,0,0,0,0,0,1,0,1.0
3,70,9550,216,0,540,756,961,756,0,1717,...,0,0,0,0,0,0,0,1,0,1.0
4,60,14260,655,0,490,1145,1145,1053,0,2198,...,0,0,0,0,0,0,0,1,0,1.0


In [224]:
#Dividing the data 
train_new= train_new.sample(frac=1).reset_index(drop=True)
train_new_train = train_new[:-100]
train_new_test = train_new[-100:]
Y_train = train_new_train["SalePrice"].values
X_train = train_new_train.drop(columns=["SalePrice"]).values
Y_test = train_new_test["SalePrice"].values
X_test = train_new_test.drop(columns=["SalePrice"]).values

print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)
print(np.mean(Y_train))

(1360, 754)
(1360,)
(100, 754)
(100,)
181804.1455882353


In [233]:
#Training using sklearn linear regression
LR = LinearRegression()
LR.fit(X_train,Y_train)
print('weight=',LR.coef_)
Y_predict = LR.predict(X_test)
print('The predictions are:',Y_predict)
print('The Real Prices are:',Y_test)

#Calculating the Mean Absolute Error

print('the Mean Absolute Error=',mean_absolute_error(Y_test,Y_predict))

weight= [-4.38307154e+01  8.78189455e-01  9.27476866e+12  9.27476866e+12
  9.27476866e+12 -9.27476866e+12  1.02499361e+13  1.02499361e+13
  1.02499361e+13 -1.02499361e+13  7.77010035e+02 -1.35635168e+03
  5.85979708e+03  6.41076487e+03 -3.06810243e+03 -6.76128011e+03
  1.66737748e+03  1.37168634e+03  5.92798994e+01  4.52214583e+03
  1.23849823e+01  2.29171245e+00  1.22033704e+01  3.54861895e+00
  1.02332114e+01  3.60335227e+01  1.99259691e+02 -2.71702334e-01
 -4.90370723e+01 -6.59520261e+02  1.83250386e+04  3.24206917e+03
  7.14857612e+03  5.67408270e+03 -7.23438354e+02  1.30416526e+04
  2.28491240e+04 -1.67786511e+03 -1.63776949e+04 -5.87121127e+03
  1.49604862e+04  1.51879440e+04 -3.42242236e+03  1.03002961e+04
  9.71633423e+02  3.32365347e+09 -4.43611316e+03  1.63820898e+03
  3.33537079e+03 -1.37939998e+05 -6.69368688e+04 -1.81750366e+03
 -1.90675494e+03  1.07488699e+05  1.10404098e+05  1.26211111e+05
  1.10953743e+05 -1.34398211e+05 -1.30732796e+05 -1.68391850e+05
 -1.29104054e+05 

In [234]:
#Training using the Linear Regression with only NumPy
W = np.linalg.pinv(X_train)@Y_train
print('weight=',W)
Y_predict = np.dot(X_test,W)
print('The predictions are:',Y_predict)
print('The Real Prices are:',Y_test)

#Calculating the Mean Absolute Error
Error = np.mean(np.abs(Y_predict - Y_test))
print('the Mean Absolute Error=', Error)

weight= [-4.41124038e+01  8.75732388e-01  1.65246379e+01  9.09320348e+00
  1.12260334e+00  2.67402448e+01  2.39526914e+01  3.42861924e+01
 -3.79485617e+01  2.02901151e+01  7.80560217e+02 -1.36105982e+03
  5.85659863e+03  6.41268171e+03 -3.06646608e+03 -6.75405771e+03
  1.66846864e+03  1.36852510e+03  5.92547341e+01  4.52844644e+03
  1.23716621e+01  2.29773116e+00  1.22159233e+01  3.59432994e+00
  1.02137980e+01  3.60504071e+01  1.99270940e+02 -2.87007699e-01
 -4.94914543e+01 -6.60261236e+02  1.83261178e+04  3.23918138e+03
  7.14809873e+03  5.67399047e+03 -7.25977663e+02  1.30389458e+04
  2.28482261e+04 -1.67596695e+03 -1.63783183e+04 -5.87210770e+03
  1.49585116e+04  1.51888725e+04 -3.42278106e+03  1.02989760e+04
  9.71581387e+02  2.00950646e-08 -4.43683918e+03  1.63818402e+03
  3.33553711e+03 -1.37942886e+05 -6.69432324e+04 -1.81741530e+03
 -1.90571057e+03  1.07484794e+05  1.10400605e+05  1.26208237e+05
  1.10950037e+05 -1.34394470e+05 -1.30728823e+05 -1.68388251e+05
 -1.29099436e+05 